In [1]:
from pyspark import SparkConf, SparkContext, StorageLevel
sc = SparkContext(conf=SparkConf()
                  .setAppName("Shortest path")
                  .setMaster("local[*]")
                  .set('spark.executor.memory', '3G')
                  .set('spark.yarn.executor.memoryOverhead', '1024'))

In [3]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist(StorageLevel.DISK_ONLY)

x = 12 # edge to start from
y = 34 # edge to finish at
d = 0  # initial distance
distances = sc.parallelize([(x, d)]).partitionBy(n)

while True:
    outer =  distances.filter(lambda i:i[1]==d)
    candidates = outer.join(forward_edges, n).map(step)
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist(StorageLevel.DISK_ONLY)
    count = new_distances.filter(lambda i: i[1] == d + 1).count()
    count_y = new_distances.filter(lambda i:i[0] == y).count()
    
    #print ("%d\t%d" % (d, count))
    d += 1
    distances = new_distances
    
    if count_y > 0:
        #print d
        break


In [4]:
path = [y]
while d > 1:
    distances_y = sc.parallelize([(y, 0)]).partitionBy(n)
    candidates_y = distances_y.join(edges, n).map(step)
    path_link = distances.filter(lambda i: i[1] == d - 1).join(candidates_y)
    prev_path_member = path_link.take(1)[0][0]
    path = [prev_path_member] + path
    y = prev_path_member
    d = d - 1
path = [x] + path
print ','.join(map(str,path))

12,422,53,52,107,20,23,274,34
